In [ ]:
import os
import cv2
from yt_dlp import YoutubeDL

# ====== CONFIGURATION ======
VIDEO_URL = "video_url"
RAW_DIR = "raw_videos"
CLIP_DIR = "clips"
START_TIMES = [5, 10, 15, 20, 25, 30, 35]  # Replace with your timestamps in seconds
CLIP_LENGTH_SEC = 5

# ====== CREATE DIRECTORIES ======
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(CLIP_DIR, exist_ok=True)

# ====== VIDEO DOWNLOADER ======
def download_video(url: str, download_dir: str) -> str:
    """
    Downloads a single video from YouTube using yt-dlp and returns the downloaded path.
    """
    output_template = os.path.join(download_dir, "%(title)s.%(ext)s")

    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'merge_output_format': 'mp4',
        'outtmpl': output_template,
        'noplaylist': True,
        'continuedl': True,
        'quiet': False,
        'no_warnings': True,
    }

    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        video_path = ydl.prepare_filename(info).replace(".webm", ".mp4")  # ensure .mp4 extension
        print(f"Downloaded video to: {video_path}")
        return video_path

In [ ]:
# ====== CLIP EXTRACTOR ======
def extract_clips(video_path: str, start_times: list, clip_length_sec: int = 5):
    """
    Extracts fixed-length video clips from a source video based on given start times (in seconds).
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Could not open video: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    clip_frames = int(round(clip_length_sec * fps))

    for idx, start_sec in enumerate(start_times):
        start_frame = int(round(start_sec * fps))
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        clip_name = f"clip_{(idx + 1):03d}.mp4"
        out_path = os.path.join(CLIP_DIR, clip_name)
        out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

        for _ in range(clip_frames):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

        out.release()
        print(f"Saved: {out_path}")

    cap.release()



In [ ]:
# ====== MAIN ======
if __name__ == "__main__":
    video_path = download_video(VIDEO_URL, RAW_DIR)
    extract_clips(video_path, START_TIMES, CLIP_LENGTH_SEC)